In [ ]:
import json
import numpy as np
from scipy.optimize import linear_sum_assignment 

distance_threshold = 0.3

your_results = 'your_results.json'

In [ ]:
with open('gt_locations.json', 'r') as f:
    data = json.load(f)

with open(f'{your_results}', 'r') as f:
    predictions = json.load(f)

In [ ]:
all_gt = []
for obj in data.keys():
    all_gt += [data[obj]]

all_pred = []
for obj in predictions['objects']:
    all_pred += [obj['position']]

In [ ]:
all_gt = np.array(all_gt)
all_pred = np.array(all_pred)

distance_matrix = []
for p in all_pred:
    dists = np.linalg.norm(all_gt - p, axis = 1) #euclidean distance
    distance_matrix += [dists]

distance_matrix = np.array(distance_matrix)

mask = distance_matrix <= distance_threshold


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(distance_matrix)
plt.xlabel('GT Locations')
plt.ylabel('Pred Locations')
plt.title('Distances')
plt.colorbar()
plt.show()

plt.imshow(mask)
plt.xlabel('GT Locations')
plt.ylabel('Pred Locations')
plt.title('Distances within threshold')
plt.colorbar()
plt.show()

In [ ]:
tp = 0  #correctly localised mug
fp = 0 #predictions that did not localise a mug
fn = 0 #mugs that were not localised

mugs_found = np.array([False for i in range(9)])
for pred_idx in range(9):
    localised = mask[pred_idx]   #did the prediction localise any of the mugs?
    
    if np.sum(localised) == 0:        # no mugs correctly localised, it is a FP
        fp += 1

    elif np.sum(localised) == 1:        # a prediction found one mug
        #check if that mug has been detected already? a mug can only be detected once
        idx = np.where(localised == True)[0][0]

        if mugs_found[idx] == True: #this is a duplicate object report
            fp += 1

        else: #this is a correct detection of a mug
            mugs_found[idx] = True
            tp += 1

    else:   #the prediction could be assigned to multiple mugs -- in this case we will start with closest mug and check all mugs under the distance
        all_idxs = np.argsort(distance_matrix[pred_idx])
        for idx in all_idxs:
            if distance_matrix[pred_idx, idx] <= distance_threshold:
                if mugs_found[idx] == False: #within threshold and not detected yet
                    mugs_found[idx] = True
                    tp += 1
                    break
                    
            else: #once we reach this point, it definitely won't be associated
                fp += 1 # must be a duplicate object report
                break
                
#fn are all mugs not associated
fn = np.sum(mugs_found == False)

print(tp, fp, fn)